<a href="https://colab.research.google.com/github/McSloats/PAM-Backdoor/blob/master/Copy_of_Copy_of_Project5_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import scipy
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Flatten
from sklearn import preprocessing
from keras import optimizers as op
from keras.utils import to_categorical
from keras.models import model_from_json
from keras.callbacks import EarlyStopping


In [12]:
##IMPORT FILE FOR DATA EXTRACTION
benign = pd.read_csv("/content/drive/Shared drives/Machine Learning Project 5/benign.csv")
malicious = pd.read_csv("/content/drive/Shared drives/Machine Learning Project 5/malicious.csv")


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
#CREATE SINGLE DATAFRAME FOR BOTH THE BENIGN AND MALICIOUS DATA WITH MIXED ROWS
benign = benign.sample(frac=1,random_state=1)
malicious = malicious.sample(frac=1,random_state=1)
data = [benign,malicious]
data = pd.concat(data)

In [0]:
##INSERT SECTION FOR DATA PRE-PROCESSING 
x = data
labels = list(x.columns.values)
data = None
benign = None 
malicious = None
x = x.fillna(0)
X = x.replace(np.inf, 0)
x = None

#IDENTIFY INDEXES OF LABELLED ENTRIES AND REMOVE THEM
temp = X.to_numpy()
index = []
count  = 0 
newArray = []
for i in temp:
  if "Src Port" in i:
    index.append(count)
  count += 1

val = 0 
for i in temp:
  if val not in index:
    newArray.append(i)
  val += 1

val = None
X = None

#SLICE 'N DICE
#CHECK TO MAKE SURE LABELLED ENTRIES ARE REMOVED
count = 0 
for i in newArray:
  if "Src Port" in i:
    print(count)
  count += 1

#REMOVE INFINITY VALUES 
array = []
for i in newArray:
  if 'Infinity' in i:
    temp = []
    for j in i:
      if j == 'Infinity':
        temp.append(0)
      else:
        temp.append(j)
    array.append(temp)
  else:
    array.append(i)
newArray = array

#Remove all zeroed entries or entries with no Flow ID 
new = []
for i in newArray:
  if i[0] != 0:
    new.append(i)
newArray = new
temp = None

df = pd.DataFrame(newArray,columns=labels)#Create new DF for data before removal and normalisation 

##DROP ATTRIBUTES FOR FEATURE SELECTION HERE 
df1 = df.drop(columns=labels[6]) # Timestamp
labels = list(df1.columns.values)
#print(list(df1.columns.values))
df1 = df1.drop(columns=[labels[1],labels[3]]) # IP Addresses. --> TO CATEGORICAL 
labels = list(df1.columns.values)
#print("Remaining Lables:\n",labels)

Y = df1[labels[-1]]
ids = df1[labels[0]]
cats = ids
X1 = df1.drop(columns=labels[0]) # Flow ID --> USE FOR PUTTING DATA TOGETHER FOR TIME SERIES
X1 = X1.drop(columns=labels[-1])
df = None
df1 = None

newY = []
for i in Y:
  if i == 'ben':
    newY.append(0)
  else:
    newY.append(1)

labelsNew = list(X1.columns.values)
for col in labelsNew:
  X1[col] = pd.to_numeric(X1[col])


In [0]:
#NORMALISE DATA
# Create a minimum and maximum processor object
scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = scaler.fit_transform(X1)

# Run the normalizer on the dataframe
df_normalized = pd.DataFrame(x_scaled,columns=labelsNew)


In [0]:
ids, _ = pd.factorize(cats)

ids = pd.DataFrame(ids,columns=['Flow_ID'])

# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
scaled = min_max_scaler.fit_transform(ids)

ids = pd.DataFrame(scaled)


In [17]:
Y = pd.DataFrame(newY,columns=['Label'])
data = pd.concat([ids,df_normalized,Y],axis=1,sort=False) 
data = data.sample(frac=1).reset_index(drop=True)
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1]
print(X)
print(Y)

[[2.46936640e-01 8.83207446e-01 5.68245975e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.70689909e-01 3.58312352e-01 5.04463264e-02 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.55951821e-01 2.12100404e-03 7.28679332e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [7.01693358e-01 7.34569314e-02 1.22072175e-03 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.64062669e-01 1.22072175e-03 4.98191806e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [5.95400392e-01 4.18554971e-02 3.81475547e-04 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
0         0
1         0
2         0
3         0
4         1
         ..
849863    1
849864    0
849865    1
849866    0
849867    1
Name: Label, Length: 849868, dtype: int64


In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
def lstm(x_train, y_train):
    #KERAS MODEL    
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = x_train.shape[1]
    #add model layers
    model.add(Embedding(100000, 96,input_length=n_cols))
    model.add(LSTM(192,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(96,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(48,return_sequences=True))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(2, activation='sigmoid'))
   
    #Optimizer
    opt = op.Adam()
    #Compile model
    model.compile(opt, loss='binary_crossentropy', 
                  metrics=['accuracy'])
   
    #TRAINING
    #Put this var in callbacks to save model after each epoch
    #checkpointer = ModelCheckpoint('model/model-{epoch:02d}.hdf5', verbose=1)

    print(model.summary())
    history = model.fit(x_train, y_train, batch_size=128, 
              validation_split=0.2, epochs=10)

    return model, history
    

In [0]:
%%timeit
model, history = lstm(x_train, y_train)


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 80, 96)            9600000   
_________________________________________________________________
lstm_13 (LSTM)               (None, 80, 192)           221952    
_________________________________________________________________
dropout_13 (Dropout)         (None, 80, 192)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 80, 96)            110976    
_________________________________________________________________
dropout_14 (Dropout)         (None, 80, 96)            0         
_________________________________________________________________
lstm_15 (LSTM)               (None, 80, 48)            27840     
_________________________________________________________________
dropout_15 (Dropout)         (None, 80, 48)           

In [0]:
%%timeit
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
from sklearn import metrics
y_pred=model.predict(x_test)
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

In [0]:
import matplotlib
